# Packages

In [351]:
import pandas as pd
import numpy as np
import json
import html
import re
import plotly.graph_objs as go
from sklearn.manifold import TSNE

import pymorphy3
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Config

In [169]:
# Doc2Vec - Experiment 1
# NROWS = 5000
# VECTOR_SIZE = 50
# MIN_COUNT = 5
# EPOCHS = 100
# ALPHA = 0.001
# MODEL_PATH = 'rjm_doc2vec_5000rows_exp1'

# Doc2Vec - Experiment 2
NROWS = 20000
VECTOR_SIZE = 50
MIN_COUNT = 5
EPOCHS = 100
ALPHA = 0.001
MODEL_PATH = 'rjm_doc2vec_20000rows_exp2'

# Data


In [170]:
def remove_html(text):
    if pd.isna(text):
        return np.nan
    text = re.sub(r"<[^>]+>", "", text)
    text = html.unescape(text)
    return text


def remove_divis(text):
    if pd.isna(text) or text.isdigit():
        return np.nan
    return text.replace("-", " ")


def map_digits(digits):
    if pd.isna(digits):
        return np.nan
    try:
        experience_num = float(digits)
        if experience_num < 0 or experience_num > 1500000:
            return np.nan
        return experience_num
    except ValueError:
        return np.nan


def map_boolean_to_yes_no(value):
    if pd.isna(value):
        return np.nan
    return "Да" if value else "Нет"


def map_country(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and "countryName" in json_data[0]:
            return json_data[0]["countryName"]
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_languageKnowledgeResume(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            languages = []
            for lang in json_data:
                if "codeLanguage" in lang and "level" in lang:
                    languages.append(
                        "{}: {}".format(lang["codeLanguage"], lang["level"])
                    )
            if languages:
                return "; ".join(languages)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_hardSkillsResume(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            hard_skills = [skill["hardSkillName"] for skill in json_data]
            return "; ".join(hard_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_softSkillsResume(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            soft_skills = [skill["softSkillName"] for skill in json_data]
            return "; ".join(soft_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan

def map_hardSkillsJob(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            hard_skills = [skill["hard_skill_name"] for skill in json_data]
            return "; ".join(hard_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan

def map_languageKnowledgeJob(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            languages = []
            for lang in json_data:
                if "code_language" in lang and "level" in lang:
                    languages.append(
                        "{}: {}".format(lang["code_language"], lang["level"])
                    )
            if languages:
                return "; ".join(languages)
    except json.JSONDecodeError:
        return np.nan
    return np.nan

def map_softSkillsJob(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            soft_skills = [skill["soft_skill_name"] for skill in json_data]
            return "; ".join(soft_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan

def map_driveLicenses(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            return ", ".join(json_data)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_educationList(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data and json_data:
            education_info = []
            for edu in json_data:
                edu_str = ""
                if "instituteName" in edu:
                    edu_str += edu["instituteName"]
                if "faculty" in edu:
                    edu_str += ", факультет: " + edu["faculty"]
                if "speciality" in edu:
                    edu_str += ", специальность: " + edu["speciality"]
                if "qualification" in edu:
                    edu_str += ", квалификация: " + edu["qualification"]
                if "graduateYear" in edu:
                    edu_str += ", год окончания: " + str(edu["graduateYear"])
                education_info.append(edu_str)

            return "; ".join(education_info)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_workExperienceList(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            work_experience_info = []
            for experience in json_data:
                experience_str = ""
                if "companyName" in experience:
                    experience_str += experience["companyName"]
                if "jobTitle" in experience:
                    experience_str += ", должность: " + experience["jobTitle"]
                if "dateFrom" in experience and "dateTo" in experience:
                    experience_str += ", период работы: {} - {}".format(
                        experience["dateFrom"][:10], experience["dateTo"][:10]
                    )
                if "demands" in experience:
                    demands_cleaned = html.unescape(
                        html.unescape(experience["demands"])
                    )
                    experience_str += ", обязанности: " + demands_cleaned.strip(
                        "<p></p>\\r\\n"
                    ).replace("<p>", "").replace("</p>", "; ")
                work_experience_info.append(experience_str)

            return "; ".join(work_experience_info)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_otherCertificates(text):
    if pd.isna(text):
        return np.nan
    if not text or text in [
        '<p><br-data-mce-bogus="1"></p>',
        "<p></p>-<p></p>",
        "<ol>-<li></li>-</ol>",
        "[]",
    ]:
        return np.nan
    else:
        cleaned_str = html.unescape(html.unescape(text))
        cleaned_str = (
            cleaned_str.replace("-", " ")
            .replace("<p>", "")
            .replace("</p>", "; ")
            .replace("<ol>", "")
            .replace("</ol>", "")
            .replace("<li>", "")
            .replace("</li>", "; ")
            .strip()
        )
        if cleaned_str:
            return cleaned_str
        else:
            return np.nan


def map_additionalEducationList(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            additional_education_info = []
            for course in json_data:
                course_str = ""
                if "organization" in course:
                    course_str += course["organization"]
                if "graduateYear" in course:
                    course_str += ", год окончания: " + str(course["graduateYear"])
                additional_education_info.append(course_str)

            return "; ".join(additional_education_info)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_birthday(birthday):
    if pd.isna(birthday):
        return np.nan
    try:
        birth_date = pd.to_datetime(birthday)
        if birth_date.year < 1900 or birth_date.year > 2023:
            return np.nan
        return birth_date.strftime("%Y-%m-%d")
    except:
        return np.nan


def map_worldskills(json_str):
    if pd.isna(json_str) or json_str == "[]":
        return np.nan
    try:
        json_data = json.loads(json_str)
        if isinstance(json_data, list) and json_data:
            skills = [
                skill["russianName"] for skill in json_data if "russianName" in skill
            ]
            if skills:
                return "; ".join(skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_accommodation_type(value):
    if pd.isna(value):
        return np.nan
    accommodation_types = {
        "ROOM": "Комната",
        "FLAT": "Квартира",
        "DORMITORY": "Общежитие",
        "HOUSE": "Дом",
    }
    return accommodation_types.get(value, value)


def map_company_business_size(value):
    if pd.isna(value):
        return np.nan
    business_sizes = {
        "SMALL": "Малый бизнес",
        "MICRO": "Микробизнес",
        "LARGE": "Крупный бизнес",
        "BIG": "Большой бизнес",
        "MIDDLE": "Средний бизнес",
    }
    return business_sizes.get(value, value)


def map_transport_compensation(value):
    if pd.isna(value):
        return np.nan
    compensation_types = {
        "PASSAGE_PAID": "Оплачивается проезд",
        "FUEL_PAID": "Оплачивается топливо",
        "AUTO": "Предоставляется автомобиль",
    }
    return compensation_types.get(value, value)


def map_retraining_capability(value):
    if pd.isna(value):
        return np.nan
    return "Возможность переобучения" if value else "Переобучение отсутствует"


def map_retraining_grant(value):
    if pd.isna(value):
        return np.nan
    grant_types = {"нет стипендии": "Нет", "есть стипендия": "Есть"}
    return grant_types.get(value, value)

## Resume


In [171]:
df_resume = pd.read_csv(
    "../datasets/cv.csv",
    encoding="UTF-8",
    on_bad_lines="skip",
    sep="|",
    low_memory=False,
    nrows=NROWS,
)
df_resume.head()

,id,candidateId,stateRegionCode,locality,localityName,birthday,gender,age,positionName,dateCreate,...,softSkills,workExperienceList,scheduleType,salary,busyType,retrainingCapability,businessTrip,languageKnowledge,relocation,innerInfo
0,06a52780-63b5-11ec-83de-839f0d9a4379,faf6f9d0-63ab-11ec-bd03-956717074877,4700000000000,4701300004000,"Ленинградская-область,-Скреблово-поселок",1963-03-01T15:00:00+0300,Мужской,60.0,водитель,2021-12-23T08:56:09+0300,...,[],"[{""companyName"": ""OAO Hовый мир"", ""dateFrom"": ...",Полный-рабочий-день,30000,Полная-занятость,Готов-к-переобучению,Готов-к-командировкам,"[{""codeLanguage"": ""Русский"", ""level"": ""Базовый...",Не-готов-к-переезду,"{""idUser"": ""9a2fe7d0-20ad-11e5-8442-1ff7059456..."
1,06a52740-cd4f-11ec-b524-4febb26dc4ec,9892ca00-b8aa-11eb-9459-a75f74b7b031,7200000000000,7200900100000,"Тюменская-область,-г.-Заводоуковск",2006-08-04T16:00:00+0400,Женский,17.0,помощник-воспитателя,2022-05-06T18:13:03+0300,...,[],[],Неполный-рабочий-день,8000,Полная-занятость,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""baa57df0-1ff3-11e5-8442-1ff7059456..."
2,06a52710-696a-11ed-a1b4-27c2a54abdc4,7b52bfa0-e760-11ec-b3c2-d9d2057e0201,4500000000000,4502100005600,"Курганская-область,-Маслянское-село",2006-01-12T15:00:00+0300,Женский,17.0,кухонный-рабочий,2022-11-21T09:59:21+0300,...,[],[],Неполный-рабочий-день,18000,Частичная-занятость,Готов-к-переобучению,Не-готов-к-командировкам,"[{""codeLanguage"": ""Русский"", ""level"": ""Базовый...",Не-готов-к-переезду,"{""idUser"": ""dc68ed30-fdd8-11ec-ba55-296f7e9e47..."
3,06a52650-8c6a-11ea-81e5-ef76bd2a03c1,61579a80-8c68-11ea-81e5-ef76bd2a03c1,7700000000000,7700000000000,г.-Москва,NaN,Мужской,NaN,Водитель-автомобиля,2020-05-02T14:42:39+0300,...,[],[],"Сменный-график,Полный-рабочий-день",30000,Полная-занятость,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""41de2a40-4d9a-11e6-a9d7-5d9e90ab95..."
4,06a52220-79f4-11ed-a8f6-27c2a54abdc4,21246300-79f3-11ed-8702-732207d240c8,5400000000000,5402600000100,"Новосибирская-область,-Усть-Тарка-село",1993-02-15T00:00:00+0300,Мужской,30.0,Сварщик,2022-12-12T11:07:30+0300,...,[],"[{""companyName"": ""ООО Агро-Флора"", ""dateFrom"":...",Полный-рабочий-день,25000,Полная-занятость,Не-готов-к-переобучению,Не-готов-к-командировкам,"[{""codeLanguage"": ""Русский"", ""level"": ""Свободн...",Не-готов-к-переезду,"{""idUser"": ""55cf48c0-2101-11e5-a317-1ff7059456..."


In [172]:
df_resume["localityName"] = df_resume["localityName"].apply(remove_divis)  # Место жительства
df_resume["relocation"] = df_resume["relocation"].apply(remove_divis)  # Готовность к переезду
df_resume["businessTrip"] = df_resume["businessTrip"].apply(remove_divis)  # Готовность к командировкам
df_resume["retrainingCapability"] = df_resume["retrainingCapability"].apply(remove_divis)  # Готовность к переобучению
df_resume["busyType"] = df_resume["busyType"].apply(remove_divis)  # Тип занятости
df_resume["scheduleType"] = df_resume["scheduleType"].apply(remove_divis)  # Желаемый график работы
df_resume["gender"] = df_resume["gender"].apply(remove_divis)  # Пол
df_resume["volunteersInspectionStatus"] = df_resume["volunteersInspectionStatus"].apply(remove_divis)  # Статус участия в движении DOBRO.RU
df_resume["narkInspectionStatus"] = df_resume["narkInspectionStatus"].apply(remove_divis)  # Статус проверки квалификации НАРК
df_resume["positionName"] = df_resume["positionName"].apply(remove_divis)  # Желаемая должность
df_resume["academicDegree"] = df_resume["academicDegree"].apply(remove_divis)  # Ученая степень
df_resume["abilympicsParticipation"] = df_resume["abilympicsParticipation"].apply(remove_divis)  # Участие в движении Абилимпикс
df_resume["salary"] = df_resume["salary"].apply(map_digits)  # Желаемая зарплата
df_resume["experience"] = df_resume["experience"].apply(map_digits)  # Опыт работы
df_resume["age"] = df_resume["age"].apply(map_digits)  # Возраст
df_resume["country"] = df_resume["country"].apply(map_country)  # Страна проживания
df_resume["languageKnowledge"] = df_resume["languageKnowledge"].apply(map_languageKnowledgeResume)  # Знание языков
df_resume["driveLicenses"] = df_resume["driveLicenses"].apply(map_driveLicenses)  # Водительские права
df_resume["hardSkills"] = df_resume["hardSkills"].apply(map_hardSkillsResume)  # Профессиональные навыки
df_resume["softSkills"] = df_resume["softSkills"].apply(map_softSkillsResume)  # Гибкие навыки
df_resume["educationList"] = df_resume["educationList"].apply(map_educationList)  # Образование
df_resume["workExperienceList"] = df_resume["workExperienceList"].apply(map_workExperienceList)  # Информация об опыте работы
df_resume["otherCertificates"] = df_resume["otherCertificates"].apply(map_otherCertificates)  # Другие сертификаты
df_resume["additionalEducationList"] = df_resume["additionalEducationList"].apply(map_additionalEducationList)  # Дополнительное образование
df_resume["birthday"] = df_resume["birthday"].apply(map_birthday)  # Дата рождения
df_resume["worldskills"] = df_resume["worldskills"].apply(map_worldskills)  # Worldskills компетенции

In [173]:
def create_resume_text(row):
    parts = []

    # Базовая информация
    if not pd.isna(row["positionName"]):
        parts.append(row["positionName"])
    if not pd.isna(row["salary"]):
        parts.append(f"Желаемая зарплата: {row['salary']}")
    if not pd.isna(row["age"]):
        parts.append(f"Возраст: {row['age']}")
    if not pd.isna(row["experience"]):
        parts.append(f"Опыт работы: {row['experience']} лет")

    # Образование и курсы
    if not pd.isna(row["educationList"]):
        parts.append(f"Образование: {row['educationList']}")
    if not pd.isna(row["additionalEducationList"]):
        parts.append(f"Дополнительное образование: {row['additionalEducationList']}")

    # Профессиональные и гибкие навыки
    if not pd.isna(row["hardSkills"]):
        parts.append(f"Профессиональные навыки: {row['hardSkills']}")
    if not pd.isna(row["softSkills"]):
        parts.append(f"Гибкие навыки: {row['softSkills']}")

    # Опыт работы
    if not pd.isna(row["workExperienceList"]):
        parts.append(f"Опыт работы: {row['workExperienceList']}")

    # Дополнительная информация
    if not pd.isna(row["otherCertificates"]):
        parts.append(f"Сертификаты и достижения: {row['otherCertificates']}")
    if not pd.isna(row["worldskills"]):
        parts.append(f"Worldskills компетенции: {row['worldskills']}")

    return " | ".join(parts)


resume_texts = df_resume.apply(create_resume_text, axis=1).tolist()

## Job


In [174]:
df_job = pd.read_csv(
    "../datasets/vacancy.csv",
    encoding="UTF-8",
    on_bad_lines="skip",
    sep="|",
    low_memory=False,
    nrows=NROWS,
)
df_job.head()

,id,academic_degree,accommodation_capability,accommodation_type,additional_premium,additional_requirements,bonus_type,measure_type,busy_type,career_perspective,...,federalDistrictCode,industryBranchName,contactList,company_name,full_company_name,company_inn,company,languageKnowledge,hardSkills,softSkills
0,47c38801-82cc-11ee-a2a9-3950de1bc4b3,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,8.0,NaN,NaN,NaN,NaN,7.709970e+09,"{""companycode"":""5147746474134"",""hr-agency"":fal...",[],[],[]
1,47a8cf85-4527-11ed-bcc0-791a818bdadb,NaN,False,NaN,NaN,<p>Адреса по вакансии Уборщик производственных...,NaN,NaN,Полная занятость,False,...,4.0,NaN,NaN,NaN,УФПС ПЕРМСКОГО КРАЯ,7.724490e+09,"{""companycode"":""4be20970-74ff-11ec-833e-4febb2...",[],[],[]
2,479f8795-6678-11ee-ac0f-e7d0d2cf29b1,NaN,True,ROOM,NaN,NaN,NaN,NaN,Полная занятость,False,...,4.0,NaN,NaN,NaN,Индивидуальный предприниматель АХТЯМОВА ДИНАРА...,4.310069e+11,"{""companycode"":""320435000033341"",""hr-agency"":f...",[],[],[]
3,478feef5-20a2-11ee-81b8-dd0276383d19,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,6.0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ХОЛЛ...",5.505043e+09,"{""companycode"":""1075505000295"",""email"":""gorin_...",[],[],[]
4,478c3f45-9325-11ed-84fd-17f530626801,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,5.0,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ СЕЛЬСКОХОЗЯЙСТВЕННОЕ ПРЕДПРИЯТИЕ...,8.907002e+09,"{""companycode"":""1028900556838"",""email"":""mujiso...",[],[],[]


In [175]:
df_job["position_responsibilities"] = df_job["position_responsibilities"].apply(remove_html)  # Обязанности на должности
df_job["position_requirements"] = df_job["position_requirements"].apply(remove_html)  # Требования к должности
df_job["additional_requirements"] = df_job["additional_requirements"].apply(remove_html)  # Дополнительные требования
df_job["other_vacancy_benefit"] = df_job["other_vacancy_benefit"].apply(remove_html)  # Другие льготы вакансии
df_job["softSkills"] = df_job["softSkills"].apply(map_softSkillsJob)  # Гибкие навыки
df_job["hardSkills"] = df_job["hardSkills"].apply(map_hardSkillsJob)  # Профессиональные навыки
df_job["required_drive_license"] = df_job["required_drive_license"].apply(map_driveLicenses)  # Требуемые водительские права
df_job["languageKnowledge"] = df_job["languageKnowledge"].apply(map_languageKnowledgeJob)  # Знание иностранных языков
df_job["accommodation_capability"] = df_job["accommodation_capability"].apply(map_boolean_to_yes_no)  # Возможность предоставления жилья
df_job["career_perspective"] = df_job["career_perspective"].apply(map_boolean_to_yes_no)  # Перспективы карьерного роста'
df_job["is_quoted"] = df_job["is_quoted"].apply(map_boolean_to_yes_no)  # Квотируемая вакансия
df_job["retraining_capability"] = df_job["retraining_capability"].apply(map_boolean_to_yes_no)  # Возможность переобучения
df_job["is_mobility_program"] = df_job["is_mobility_program"].apply(map_boolean_to_yes_no)  # Программа мобильности
df_job["need_medcard"] = df_job["need_medcard"].apply(map_boolean_to_yes_no)  # Необходимость медицинской книжки
df_job["is_uzbekistan_recruitment"] = df_job["is_uzbekistan_recruitment"].apply(map_boolean_to_yes_no)  # Возможность найма граждан Узбекистана
df_job["accommodation_type"] = df_job["accommodation_type"].apply(map_accommodation_type)  # Тип предоставляемого жилья
df_job["company_business_size"] = df_job["company_business_size"].apply(map_company_business_size)  # Размер компании
df_job["transport_compensation"] = df_job["transport_compensation"].apply(map_transport_compensation)  # Компенсация транспортных расходов
df_job["retraining_grant"] = df_job["retraining_grant"].apply(map_retraining_grant)  # Наличие стипендии во время переобучения
df_job["retraining_grant_value"] = df_job["retraining_grant_value"].apply(map_digits)  # Размер стипендии во время переобучения
df_job["salary_min"] = df_job["salary_min"].apply(map_digits)  # Минимальная заработная плата
df_job["salary_max"] = df_job["salary_max"].apply(map_digits)  # Максимальная заработная плата
df_job["additional_premium"] = df_job["additional_premium"].apply(map_digits)  # Дополнительная премия

# 'full_company_name': 'Полное название компании'
# 'vacancy_address': 'Адрес вакансии'
# 'education': 'Образование'
# 'education_speciality': 'Специальность по образованию'
# 'required_certificates': 'Требуемые сертификаты'
# 'professionalSphereName': 'Название профессиональной сферы'
# 'industryBranchName': 'Название отрасли промышленности'
# 'academic_degree': 'Ученая степень'
# 'vacancy_name': 'Наименование вакансии'
# 'required_experience': 'Требуемый опыт работы
# 'busy_type': 'Тип занятости'
# 'schedule_type': 'Тип графика работы'

In [176]:
def create_job_text(row):
    parts = []

    # Job Details
    if not pd.isna(row["vacancy_name"]):
        parts.append(f"{row['vacancy_name']}")
    if not pd.isna(row["salary"]):
        parts.append(f"Заработная плата: {row['salary_min']}")

    if not pd.isna(row["position_responsibilities"]):
        parts.append(f"Обязанности: {row['position_responsibilities']}")
    if not pd.isna(row["position_requirements"]):
        parts.append(f"Требования: {row['position_requirements']}")
    if not pd.isna(row["additional_requirements"]):
        parts.append(f"Дополнительные требования: {row['additional_requirements']}")

    if not pd.isna(row["hardSkills"]):
        parts.append(f"Профессиональные навыки: {row['hardSkills']}")
    if not pd.isna(row["softSkills"]):
        parts.append(f"Гибкие навыки: {row['softSkills']}")

    return " | ".join(parts)


job_texts = df_job.apply(create_job_text, axis=1).tolist()

# Text Preprocessing


In [177]:
# Расширенный список стоп-слов
sw = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"

sw.extend(additional_sw.split())
sw.extend(pronouns.split())
sw.extend(conjunctions.split())
sw.extend(digits.split())
sw.extend(modal_words.split())
sw.extend(particles.split())
sw.extend(prepositions.split())
sw.extend(evaluative.split())
sw = list(set(sw))

In [178]:
pattern = re.compile("[^а-яА-Яa-zA-Z0-9\-.,;]+")
morph = pymorphy3.MorphAnalyzer()

def tokenize_and_preprocess(text, stopwords):
    text = text.lower()
    text = pattern.sub(" ", text)
    tokens = simple_preprocess(text, deacc=True)
    tokens = [
        morph.parse(token)[0].normal_form
        for token in tokens
        if token not in stopwords and len(token) > 2
    ]
    return tokens

In [179]:
%%time

processed_resume_texts = [tokenize_and_preprocess(text, sw) for text in resume_texts]

CPU times: user 1min 2s, sys: 296 ms, total: 1min 2s
Wall time: 1min 2s


In [180]:
%%time

processed_job_texts = [tokenize_and_preprocess(text, sw) for text in job_texts]

CPU times: user 1min 8s, sys: 481 ms, total: 1min 9s
Wall time: 1min 9s


# Doc2Vec


In [181]:
tagged_data = [
    TaggedDocument(words=text, tags=[f"resume_{i}"])
    for i, text in enumerate(processed_resume_texts)
]
tagged_data += [
    TaggedDocument(words=text, tags=[f"job_{i}"])
    for i, text in enumerate(processed_job_texts)
]

In [182]:
# Model initialization
model = Doc2Vec(
    vector_size=VECTOR_SIZE,
    min_count=MIN_COUNT,
    epochs=EPOCHS,
    alpha=ALPHA,
)
model.build_vocab(tagged_data)

keys = model.wv.key_to_index.keys()
len(keys) # length of vocabulary keys

13564

In [183]:
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

model.save(f"../models/{MODEL_PATH}.model")
print("Model saved")

Training epoch 1/100
Training epoch 2/100
Training epoch 3/100
Training epoch 4/100
Training epoch 5/100
Training epoch 6/100
Training epoch 7/100
Training epoch 8/100
Training epoch 9/100
Training epoch 10/100
Training epoch 11/100
Training epoch 12/100
Training epoch 13/100
Training epoch 14/100
Training epoch 15/100
Training epoch 16/100
Training epoch 17/100
Training epoch 18/100
Training epoch 19/100
Training epoch 20/100
Training epoch 21/100
Training epoch 22/100
Training epoch 23/100
Training epoch 24/100
Training epoch 25/100
Training epoch 26/100
Training epoch 27/100
Training epoch 28/100
Training epoch 29/100
Training epoch 30/100
Training epoch 31/100
Training epoch 32/100
Training epoch 33/100
Training epoch 34/100
Training epoch 35/100
Training epoch 36/100
Training epoch 37/100
Training epoch 38/100
Training epoch 39/100
Training epoch 40/100
Training epoch 41/100
Training epoch 42/100
Training epoch 43/100
Training epoch 44/100
Training epoch 45/100
Training epoch 46/1

## Visualisation

In [237]:
def plot_embeddings(model, tagged_docs):
    # Получение векторов и меток
    vectors = [model.infer_vector(doc.words) for doc in tagged_docs]
    labels = [doc.tags[0] for doc in tagged_docs]

    # Конвертация списка векторов в массив NumPy
    vectors_np = np.array(vectors)

    # Применение t-SNE для снижения размерности
    tsne = TSNE(n_components=2, random_state=0)
    vectors_2d = tsne.fit_transform(vectors_np)

    traces = []
    for point, label in zip(vectors_2d, labels):
        color = 'red' if 'resume' in label else 'blue'
        trace = go.Scatter(
            x=[point[0]], 
            y=[point[1]],
            mode='markers+text',
            marker=dict(color=color),
            text=label,
            textposition="bottom center"
        )
        traces.append(trace)

    # Создание графика
    layout = go.Layout(title="Embeddings Visualization")
    fig = go.Figure(data=traces, layout=layout)

    fig.show()


In [238]:
plot_embeddings(model, tagged_data[:1000]) # Визуализация первой 1000 документов

## Ranking

In [192]:
def preprocess_text(text):
    text = text.lower()
    pattern = re.compile("[^а-яА-Яa-zA-Z0-9\-.,;]+")
    text = pattern.sub(" ", text)
    text = ' '.join(text.split())
    return text

In [193]:
def rank_resumes_for_job(model, job_desc, resumes, threshold=0.5):
    job_desc = preprocess_text(job_desc)
    job_vector = model.infer_vector(job_desc.split())
    ranked_resumes = []
    for resume in resumes:
        resume = preprocess_text(resume)
        resume_vector = model.infer_vector(resume.split())
        similarity = np.dot(job_vector, resume_vector) / (np.linalg.norm(job_vector) * np.linalg.norm(resume_vector))
        if similarity >= threshold:
            ranked_resumes.append((resume, similarity))
    ranked_resumes.sort(key=lambda x: x[1], reverse=True)
    return [(resume, round(similarity * 100, 2)) for resume, similarity in ranked_resumes]

In [194]:
def rank_jobs_for_resume(model, resume_desc, jobs, threshold=0.5):
    resume_desc = preprocess_text(resume_desc)
    resume_vector = model.infer_vector(resume_desc.split())
    ranked_jobs = []
    for job in jobs:
        job = preprocess_text(job)
        job_vector = model.infer_vector(job.split())
        similarity = np.dot(resume_vector, job_vector) / (np.linalg.norm(resume_vector) * np.linalg.norm(job_vector))
        if similarity >= threshold:
            ranked_jobs.append((job, similarity))
    ranked_jobs.sort(key=lambda x: x[1], reverse=True)
    return [(job, round(similarity * 100, 2)) for job, similarity in ranked_jobs]

### Example

In [253]:
model = Doc2Vec.load(f"../models/rjm_doc2vec_exp2.model")

In [254]:
resumes = [
    "Опытный системный аналитик с пятилетним стажем работы в крупной IT-компании. Основные обязанности включали анализ бизнес-процессов, сбор требований от заказчиков, разработку технических заданий и координацию разработчиков. Умею работать с SQL, UML, BPMN. Опыт работы с Agile и Scrum. В прошлом году успешно реализовал проект автоматизации процессов для крупного ритейлера, что позволило клиенту увеличить эффективность работы на 20%. Отличные коммуникативные навыки, способность работать в команде и под высоким давлением.",
    "Менеджер по продажам с более чем десятилетним опытом в сфере B2B и B2C. Специализируюсь на продажах в области финансовых услуг и страхования. Отлично разбираюсь в продуктах компании, умею находить подход к различным типам клиентов. Имею опыт организации и проведения презентаций, участия в переговорах и заключения крупных сделок. В прошлом году лично привлек более 30 новых корпоративных клиентов, что способствовало росту общего оборота компании на 15%. Владею техниками активных продаж, хорошо ориентируюсь в CRM-системах.",
    "Главный инженер с более чем 15-летним опытом работы в сфере машиностроения. Ответственен за разработку и внедрение новых технологий, управление инженерными командами, контроль качества продукции. Имею опыт работы с CAD/CAM системами, знаком с современными методами производства и автоматизации. На моем счету множество успешно реализованных проектов, в том числе разработка и запуск новой линии по производству автомобильных компонентов. Отличные лидерские качества, способность принимать быстрые и эффективные решения в критических ситуациях.",
    "Веб-разработчик с опытом работы более 8 лет. Специализируюсь на front-end и back-end разработке. Владею HTML, CSS, JavaScript, а также фреймворками React и Node.js. Работал над созданием и поддержкой нескольких крупных веб-платформ, включая интернет-магазины и корпоративные сайты. Имею опыт работы в Agile-командах, умею четко следовать срокам и требованиям заказчика. За последний год разработал и внедрил новый интерфейс для онлайн-магазина, что привело к увеличению конверсии на 30%.",
    "Творческий и инновационный графический дизайнер с 7-летним опытом работы в рекламных агентствах и дизайн-студиях. Специализируюсь на создании брендинга, рекламных материалов, веб-дизайна и UX/UI. Владею Adobe Photoshop, Illustrator, InDesign. Мои работы отличаются оригинальностью и уникальностью, я всегда стремлюсь найти нестандартные решения для задач клиентов. Разработал брендинг и фирменный стиль для нескольких стартапов, что помогло им выделиться на рынке и привлечь новых клиентов. Умею работать в сжатые сроки, легко адаптируюсь к новым требованиям и тенденциям в дизайне."
]

jobs = [
    "Требуется опытный руководитель отдела продаж для работы в крупной торговой компании. Обязанности: разработка и реализация стратегии продаж, управление командой менеджеров, анализ рынка и конкурентов, участие в переговорах на высоком уровне. Требования: опыт работы на аналогичной должности не менее 5 лет, отличные навыки управления и коммуникации, знание современных технологий продаж. Предлагаем стабильную работу в динамично развивающейся компании, конкурентоспособную заработную плату и возможности для профессионального роста.",
    "ИТ-компания ищет талантливого программиста 1C для разработки и поддержки корпоративных приложений. Основные задачи: разработка и оптимизация программных решений на платформе 1C, работа с клиентами по техническим заданиям, тестирование и документирование программного обеспечения. Требуется знание 1C:Предприятие 8, опыт программирования и понимание бизнес-процессов. Мы предлагаем работу в комфортабельном офисе, дружный коллектив и достойную заработную плату.",
    "В маркетинговое агентство требуется креативный и амбициозный маркетолог. Обязанности: разработка маркетинговых стратегий, проведение рекламных кампаний, анализ эффективности маркетинговых действий, работа с социальными сетями и контентом. Требуется опыт в маркетинге не менее 3 лет, умение работать в команде, знание инструментов интернет-маркетинга. Мы предлагаем работу в динамичной среде, возможность реализовать свои идеи и конкурентоспособную заработную плату.",
    "Требуется инженер-конструктор в инженерно-проектное бюро. Задачи: разработка проектной документации, расчеты и моделирование в CAD-системах, сопровождение проектов на всех этапах. Требования: высшее техническое образование, опыт работы в проектировании, знание AutoCAD и других CAD-систем. Предлагаем интересные проекты, работу в команде профессионалов, стабильную заработную плату и возможности для карьерного роста.",
    "В рекламное агентство требуется творческий копирайтер для создания текстов различной тематики. Обязанности: написание рекламных и PR-текстов, создание контента для сайтов и социальных сетей, работа над креативными концепциями. Требуется опыт написания текстов, креативность, отличное владение русским языком. Мы предлагаем возможность реализации творческого потенциала, работу над интересными проектами и достойную оплату труда."
]

In [350]:
rank_resumes_for_job(model, jobs[3], resumes) # вакансия веб-разработчика

[('веб-разработчик с опытом работы более 8 лет. специализируюсь на front-end и back-end разработке. владею html, css, javascript, а также фреймворками react и node.js. работал над созданием и поддержкой нескольких крупных веб-платформ, включая интернет-магазины и корпоративные сайты. имею опыт работы в agile-командах, умею четко следовать срокам и требованиям заказчика. за последний год разработал и внедрил новый интерфейс для онлайн-магазина, что привело к увеличению конверсии на 30 .',
  71.49),
 ('главный инженер с более чем 15-летним опытом работы в сфере машиностроения. ответственен за разработку и внедрение новых технологий, управление инженерными командами, контроль качества продукции. имею опыт работы с cad cam системами, знаком с современными методами производства и автоматизации. на моем счету множество успешно реализованных проектов, в том числе разработка и запуск новой линии по производству автомобильных компонентов. отличные лидерские качества, способность принимать быстр

In [332]:
rank_jobs_for_resume(model, resumes[4], jobs) # резюме дизайнера

[('в рекламное агентство требуется творческий копирайтер для создания текстов различной тематики. обязанности написание рекламных и pr-текстов, создание контента для сайтов и социальных сетей, работа над креативными концепциями. требуется опыт написания текстов, креативность, отличное владение русским языком. мы предлагаем возможность реализации творческого потенциала, работу над интересными проектами и достойную оплату труда.',
  52.26)]

# References

[CV Job Matching using Doc2Vec](https://github.com/kirudang/CV-Job-matching/blob/main/Matching_Algorithm.ipynb)